# ICS2123 2025.1 - Tarea 2

# Importamos algunas librerías

In [31]:
import numpy as np
import math
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

# Función que permite armar un diccionario vacío con las probabilidades de transición y la lista de estados (NO MODIFICAR)

In [32]:
def armar_estados(lista_estaciones, lista_capacidad):
    estados = []
    for i in lista_capacidad:
        for j in lista_estaciones:   
            estados.append((i, j, 0))
            estados.append((i, j, 1))
    
    # Armar diccionario con las probabilidades de transición, seteadas en 0:
    probabilidades_transicion = {}
    for s1 in estados:
        for s2 in estados:
            probabilidades_transicion[(s1, s2)] = 0
    
    return estados, probabilidades_transicion




    


# PARÁMETROS DE ESTADOS (NO MODIFICAR)

In [33]:
cantidad_estaciones = 15
capacidad = 10
estaciones = [i for i in range(1, cantidad_estaciones+1)]
capacidades = [i for i in range(0, capacidad+1)]

print(f"Estaciones: {estaciones}")
print(f"Capacidades: {capacidades}")

estados, probabilidades_transicion = armar_estados(estaciones, capacidades)

print(f"Estados: {estados}")
#print(f"Probabilidades de transición: {probabilidades_transicion}")
print(f"Cantidad Estados: {len(estados)}")


Estaciones: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Capacidades: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Estados: [(0, 1, 0), (0, 1, 1), (0, 2, 0), (0, 2, 1), (0, 3, 0), (0, 3, 1), (0, 4, 0), (0, 4, 1), (0, 5, 0), (0, 5, 1), (0, 6, 0), (0, 6, 1), (0, 7, 0), (0, 7, 1), (0, 8, 0), (0, 8, 1), (0, 9, 0), (0, 9, 1), (0, 10, 0), (0, 10, 1), (0, 11, 0), (0, 11, 1), (0, 12, 0), (0, 12, 1), (0, 13, 0), (0, 13, 1), (0, 14, 0), (0, 14, 1), (0, 15, 0), (0, 15, 1), (1, 1, 0), (1, 1, 1), (1, 2, 0), (1, 2, 1), (1, 3, 0), (1, 3, 1), (1, 4, 0), (1, 4, 1), (1, 5, 0), (1, 5, 1), (1, 6, 0), (1, 6, 1), (1, 7, 0), (1, 7, 1), (1, 8, 0), (1, 8, 1), (1, 9, 0), (1, 9, 1), (1, 10, 0), (1, 10, 1), (1, 11, 0), (1, 11, 1), (1, 12, 0), (1, 12, 1), (1, 13, 0), (1, 13, 1), (1, 14, 0), (1, 14, 1), (1, 15, 0), (1, 15, 1), (2, 1, 0), (2, 1, 1), (2, 2, 0), (2, 2, 1), (2, 3, 0), (2, 3, 1), (2, 4, 0), (2, 4, 1), (2, 5, 0), (2, 5, 1), (2, 6, 0), (2, 6, 1), (2, 7, 0), (2, 7, 1), (2, 8, 0), (2, 8, 1), (2, 9, 0), (2, 9, 

# PARAMETROS DE SUBIDA Y BAJADA DE PASAJEROS (NO MODIFICAR)

In [34]:
def armar_parametros_variables_aleatorias(lista_estaciones, capacidad_maxima):
    np.random.seed(2123)
    parametros_subida = {}
    parametros_bajada = {}
    for i in lista_estaciones:
        parametros_bajada[i] = round(np.random.uniform(0.1, 0.9),1)
        if i != lista_estaciones[-1]:
            parametros_subida[i, i+1] = int(round(np.random.uniform(2, capacidad_maxima),0))
        if i != lista_estaciones[0]:
            parametros_subida[i, i-1] =  int(round(np.random.uniform(2, capacidad_maxima),0))
    
    return parametros_subida, parametros_bajada


In [35]:
parametros_subida, parametros_bajada = armar_parametros_variables_aleatorias(estaciones, max(capacidades))

print(f"Parametros de subida: {parametros_subida}")
print(f"Parametros de bajada: {parametros_bajada}")

# Subida será una variable aleatoria uniforme discreta que toma valores entre 0 y un parámetro de subida que depende de la estación actual y la siguiente
# Bajada será una variable aleatoria binomial que tiene como total la cantidad de personas actuales en el vagón, y como probabilidad un párametro de bajada que depende de la estación actual y la siguiente

Parametros de subida: {(1, 2): 9, (2, 3): 3, (2, 1): 5, (3, 4): 5, (3, 2): 9, (4, 5): 7, (4, 3): 7, (5, 6): 9, (5, 4): 8, (6, 7): 9, (6, 5): 5, (7, 8): 5, (7, 6): 2, (8, 9): 6, (8, 7): 7, (9, 10): 5, (9, 8): 4, (10, 11): 7, (10, 9): 6, (11, 12): 7, (11, 10): 6, (12, 13): 9, (12, 11): 4, (13, 14): 9, (13, 12): 5, (14, 15): 9, (14, 13): 4, (15, 14): 8}
Parametros de bajada: {1: 0.3, 2: 0.4, 3: 0.6, 4: 0.9, 5: 0.5, 6: 0.6, 7: 0.4, 8: 0.7, 9: 0.7, 10: 0.3, 11: 0.8, 12: 0.2, 13: 0.7, 14: 0.7, 15: 0.7}


# Función que permite obtener probabilidad de subida al viajar de una estación i a una j. (NO MODIFICAR)

In [36]:
def obtener_probabilidad_exacta_subida(parametro_subida, parametro_bajada, cantidad_actual,  cantidad_nueva, estacion_actual, estacion_siguiente):
    min_uniforme = 0
    max_uniforme = parametro_subida[(estacion_actual, estacion_siguiente)]
    prob_binomial = parametro_bajada[estacion_siguiente]
    prob_total = 0
    for i in range(cantidad_actual+1):
        if i + cantidad_nueva - cantidad_actual > max_uniforme:
            break
        prob_uniforme_discreta_i = 1/(max_uniforme - min_uniforme +1)
        prob_binomial_i = math.comb(cantidad_actual, i) * (prob_binomial**i) * ((1-prob_binomial)**(cantidad_actual-i))
        prob_total += prob_uniforme_discreta_i * prob_binomial_i
    return prob_total


# Función que permite obtener la probabilidad de que el vagón se llene. (NO MODIFICAR)

In [37]:
def probabilidad_hasta_m(parametro_subida, parametro_bajada, cantidad_actual, capacidad_maxima, estacion_actual, estacion_siguiente):
    min_uniforme = 0
    max_uniforme = parametro_subida[(estacion_actual, estacion_siguiente)]
    prob_binomial = parametro_bajada[estacion_siguiente]
    prob_total = 0
    for i in range(cantidad_actual+1):
        prob_uniforme_discreta_i = (1- (capacidad_maxima - cantidad_actual +i  - min_uniforme)/(max_uniforme - min_uniforme +1))
        prob_binomial_i = math.comb(cantidad_actual, i) * (prob_binomial**i) * ((1-prob_binomial)**(cantidad_actual-i))
        if prob_binomial_i < 0 or prob_uniforme_discreta_i < 0:
            break
        prob_total += prob_uniforme_discreta_i * prob_binomial_i


    return prob_total


# Función que permite obtener la probabilidad de que el vagón baje la cantidad de personas al viajar de una estación a otra. (NO MODIFICAR)

In [38]:
def obtener_probabilidad_exacta_bajada(parametro_subida, parametro_bajada, cantidad_actual,  cantidad_nueva, estacion_actual, estacion_siguiente):
    min_uniforme = 0
    max_uniforme = parametro_subida[(estacion_actual, estacion_siguiente)]
    prob_binomial = parametro_bajada[estacion_siguiente]
    prob_total = 0
    for i in range(max_uniforme+1):
        
        if i > cantidad_nueva:
            break
        prob_uniforme_discreta_i = 1/(max_uniforme - min_uniforme +1)
        prob_binomial_i = math.comb(cantidad_actual, cantidad_actual - cantidad_nueva + i) * (prob_binomial**(cantidad_actual - cantidad_nueva + i)) * ((1-prob_binomial)**(cantidad_nueva-i))
        prob_total += prob_uniforme_discreta_i * prob_binomial_i
    
    return prob_total

# Permite rellenar la matriz de probabilidades de transición. (NO MODIFICAR)

In [39]:
def armar_matriz_de_probabilidad_transicion(estados, probabilidades_transicion, parametros_subida, parametros_bajada, capacidad_maxima):
    lista_estados_out = []
    
    for s1 in estados:
        c, e, d = s1

        if d== 0 and e == cantidad_estaciones:
            lista_estados_out.append(s1)
            continue
            
        if d == 1 and e == 1:
            lista_estados_out.append(s1)
            continue

        for c_prima in capacidades:
            
            if d == 1:
                if e == 2:
                    proximo_e = 1
                    proximo_d = 0
                elif e > 2:
                    proximo_e = e - 1
                    proximo_d = 1
            else:
                if e == cantidad_estaciones-1:
                    proximo_e = cantidad_estaciones
                    proximo_d = 1
                elif e < cantidad_estaciones-1:
                    proximo_e = e + 1
                    proximo_d = 0
                     

                

            s2 = (c_prima, proximo_e, proximo_d)
            if c_prima < c:
                prob = obtener_probabilidad_exacta_bajada(parametros_subida, parametros_bajada, c, c_prima, e, proximo_e)
                
                probabilidades_transicion[(s1, s2)] = prob
            else:
                if c_prima - c > parametros_subida[(e, proximo_e)]:
                    break
                if c_prima == capacidad_maxima:
                    prob = probabilidad_hasta_m(parametros_subida, parametros_bajada, c, capacidad_maxima, e, proximo_e)
                    probabilidades_transicion[(s1, s2)] = prob

                else:
                    prob = obtener_probabilidad_exacta_subida(parametros_subida, parametros_bajada, c, c_prima, e, proximo_e)
                    probabilidades_transicion[(s1, s2)] = prob
    
    # Quitar los estados out de la lista de estados (estadps inancanzables) y de las probabilidades de transición:
    for s in lista_estados_out:
        estados.remove(s)
        for s2 in estados:
            probabilidades_transicion.pop((s, s2))
            probabilidades_transicion.pop((s2, s))

    return probabilidades_transicion, estados

                
                    

In [40]:
probabilidades_transicion, estados_nuevos = armar_matriz_de_probabilidad_transicion(estados, probabilidades_transicion, parametros_subida, parametros_bajada, capacidad)

print(f"Probabilidades de transición: {probabilidades_transicion}")

# Revisar que la suma de las probabilidades de transición para cada estado sea 1:
for s1 in estados:
    suma = 0
    for s2 in estados:
        if probabilidades_transicion[(s1, s2)] != 0:
            print(f"La probabilidad de ir de {s1} a {s2} es {probabilidades_transicion[(s1, s2)]}")
        suma += probabilidades_transicion[(s1, s2)]
    
    print(f"La suma de las probabilidades de transición para el estado {s1} es {round(suma,3)}")
    # Normalizar (puesto que hay aproximaciones que hacen que la suma no sea 1, pero la diferencia es muy pequeña):
    for s2 in estados:
        
        probabilidades_transicion[(s1, s2)] = probabilidades_transicion[(s1, s2)]/suma

    
    #print(f"La suma de las probabilidades de transición para el estado {s1} es {round(suma,3)}")




Probabilidades de transición: {((0, 1, 0), (0, 1, 0)): 0, ((0, 1, 0), (0, 2, 0)): 0.1, ((0, 1, 0), (0, 2, 1)): 0, ((0, 1, 0), (0, 3, 0)): 0, ((0, 1, 0), (0, 3, 1)): 0, ((0, 1, 0), (0, 4, 0)): 0, ((0, 1, 0), (0, 4, 1)): 0, ((0, 1, 0), (0, 5, 0)): 0, ((0, 1, 0), (0, 5, 1)): 0, ((0, 1, 0), (0, 6, 0)): 0, ((0, 1, 0), (0, 6, 1)): 0, ((0, 1, 0), (0, 7, 0)): 0, ((0, 1, 0), (0, 7, 1)): 0, ((0, 1, 0), (0, 8, 0)): 0, ((0, 1, 0), (0, 8, 1)): 0, ((0, 1, 0), (0, 9, 0)): 0, ((0, 1, 0), (0, 9, 1)): 0, ((0, 1, 0), (0, 10, 0)): 0, ((0, 1, 0), (0, 10, 1)): 0, ((0, 1, 0), (0, 11, 0)): 0, ((0, 1, 0), (0, 11, 1)): 0, ((0, 1, 0), (0, 12, 0)): 0, ((0, 1, 0), (0, 12, 1)): 0, ((0, 1, 0), (0, 13, 0)): 0, ((0, 1, 0), (0, 13, 1)): 0, ((0, 1, 0), (0, 14, 0)): 0, ((0, 1, 0), (0, 14, 1)): 0, ((0, 1, 0), (0, 15, 1)): 0, ((0, 1, 0), (1, 1, 0)): 0, ((0, 1, 0), (1, 2, 0)): 0.1, ((0, 1, 0), (1, 2, 1)): 0, ((0, 1, 0), (1, 3, 0)): 0, ((0, 1, 0), (1, 3, 1)): 0, ((0, 1, 0), (1, 4, 0)): 0, ((0, 1, 0), (1, 4, 1)): 0, ((0, 1, 0

# Función que utiliza DFS y memoización para contar caminos. A partir de este código, deben inspirarse para implementar lo que se solicita en el apartado b (NO MODIFICAR)

In [ ]:
# Función para contar e imprimir caminos válidos en una grilla con obstáculos usando DFS + memoización
def contar_e_imprimir_caminos(m, n, celdas_bloqueadas):
    bloqueadas = set(celdas_bloqueadas)
    memo = {}

    def dfs(i, j, path):
        # Si estamos fuera de la grilla o en una celda bloqueada
        if i >= m or j >= n or (i, j) in bloqueadas:
            return 0

        # Si llegamos al destino, imprimir el camino
        if (i, j) == (m-1, n-1):
            path.append((i, j))
            print("→ Camino:", path)
            path.pop()
            return 1

        if (i, j) in memo:
            # No usamos memo aquí porque los caminos pueden variar incluso con el mismo número total
            # Este memo se puede usar solo para conteo, no para caminos explícitos
            pass

        path.append((i, j))
        total = 0
        total += dfs(i+1, j, path)  # Mover abajo
        total += dfs(i, j+1, path)  # Mover derecha
        path.pop()

        memo[(i, j)] = total
        return total

    total_caminos = dfs(0, 0, [])
    print(f"\n🔢 Total de caminos posibles: {total_caminos}")

# Ejemplo de uso
m = 5
n = 5
celdas_bloqueadas = [(1, 1), (3,2)]

contar_e_imprimir_caminos(m, n, celdas_bloqueadas)

→ Camino: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]
→ Camino: [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (4, 1), (4, 2), (4, 3), (4, 4)]
→ Camino: [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (4, 2), (4, 3), (4, 4)]
→ Camino: [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (4, 3), (4, 4)]
→ Camino: [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 4)]
→ Camino: [(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (4, 1), (4, 2), (4, 3), (4, 4)]
→ Camino: [(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (3, 2), (4, 2), (4, 3), (4, 4)]
→ Camino: [(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (3, 2), (3, 3), (4, 3), (4, 4)]
→ Camino: [(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (3, 2), (3, 3), (3, 4), (4, 4)]
→ Camino: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
→ Camino: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (3, 3), (4, 3), (4, 4)]
→ Camino: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (3, 3), (3, 4), (4, 4)]
→ Ca

## Inciso b)

## Inciso c)

## Inciso d)